## Noisy labeling 
### EMNIST DATASET

연구자: 정회희 

### 연구 배경

우리가 학습하는 데이터는 사실 하나의 절대적인 oracle이 라벨링한 것이 아니라 다수의 서로 다른 가치관을 가진 사람들이 서로 다른 기준으로 때로는 실수도 하면서 라벨링한 결과물이다. 때문에 우리가 학습에 사용하는 (몇몇의 labeler들이 관측/라벨링한) 데이터셋의 라벨은 애매성이 있고, 이를 random 또는 noisy하다고 가정할 수 있을 것이다. 

하지만 문제는 우리가 현재 가지고 있는 데이터의 randomness가 학습 데이터에서 잘못 라벨링된 것들이 학습에 악영향을 미칠 수도 있다는 것이다. 잘못된 라벨을 가지고 학습하게 된다면 처음 본 낯선 데이터의 경우에는 완전히 잘못 학습되어 반대로 라벨링된 대답을 할 수도 있고, 그 전에 비슷하게 생긴 데이터를 여러번 관측했더라도 이들이 라벨 노이즈 때문에 서로 다른 라벨을 갖고있어 대답에 확실성이 떨어뜨리게 될 것이다.

이러한 문제들은 이론상으로만 존재하는 것이 아니라 실제로 인공신경망을 학습하다보면 자주 관측되는 문제이다. 때문에 noise의 정도가 학습에 얼마나 큰 영향을 미치는지, 이를 해결하기 위해서는 어떤 방법론이 있는지, 앞에서 제시한 방법론들이 얼마만큼 효과를 볼 수 있는지 등등을 심층적으로 분석해보고자 한다.

### 문제 설정

- 원본 데이터를 ground truth라고 가정함.
- 원본 데이터의 training set의 라벨을 의도적으로 바꿔 인공신경망을 학습한다.
    - 라벨을 바꾸는 policy는 여러가지가 있겠지만 이 실험에서는 최대한 단순화하기 위해서 random하게 진행하였다.
    - random하게 라벨을 바꾸기 위해서 p값을 도입함. p 확률로는 라벨이 바뀌고, 1-p 확률로는 라벨이 바뀌지 않음.
    - $$
f(y)=
\begin{cases}
y & \text{w.p.} & 1 - p\\
\bar{y} \in \mathbb{Y} \backslash y & \text{w.p.} & p \\
\end{cases}
$$
    - 1-p 확률로 라벨이 바뀔 때는 마찬가지로 여러 방법이 있겠지만, 기존의 라벨 y가 아닌 나머지 라벨에 대해서 uniform하게 바뀌게 함.
    - 한 번 노이즈가 추가되고 난 후 라벨은 바뀌지 않음. (하나의 이미지가 가지는 라벨이 learning하는 중간에는 바뀌지 않음.)
    - 라벨을 관측할 때마다 바뀌는 세팅과 구분. (하나의 이미지가 가지는 라벨이 learning하는 중간에 바뀔 수 있음. e.g. minibatch에 들어갈 때마다)
        - 이 경우는 좀 더 쉬운 문제가 됨. (동일한 image에 대해서 epoch를 많이 거치게 되면 uniform noise가 상쇄될 수 있기 때문)
        - 하지만 이런 경우는 실제 상황과 동떨어져 있음.
        - refer: https://blog.lunit.io/2017/08/08/deep-learning-is-robust-to-massive-label-noise/
- 원본 데이터의 testing set의 라벨은 원본 그대로 (ground truth 상태로) 검증 단계에서 사용한다.

### 데이터 선택
- 데이터 수가 적을 경우 문제 해결이 지나치게 쉬울 수 있음. (e.g. 모든 데이터셋을 일일이 검토) 따라서 우리가 직면하고 있는 문제와 가깝게 하기 위해서 데이터 수가 많아야 함.
- class imbalance에 대한 효과가 성능 비교에 영향을 미치지 않기 위해서 class 분포가 uniform한 데이터여야 함.
- 따라서 balanced EMNIST dataset으로 함.

#### EMIST DATASET
refer: https://www.kaggle.com/crawford/emnist
- 0에서 9까지 숫자를 분류하는 MNIST의 원본 데이터 NIST에서 만들어진 데이터셋
- 0부터 9까지 숫자와 알파벳을 가지고 있음.
- 종류
    - By Class: 0~9(10), 대문자(26), 소문자(26) 알파벳
    - By Merge: 0~9(10), 대문자(26), 소문자(11) 알파벳
        - 소문자 알파벳에는 대문자와 지나치게 헷깔리는 것들을 제외함. 예를 들어서 소문자 c와 대문자 C처럼 크기만 다른 경우.
        - 11개 소문자는 다음과 같다:a,b,d,e,f,g,h,n,q,r,t
    - Balance: 0~9(10), 대문자(26), 소문자(11) 알파벳
        - By Merge 데이터셋에서 class간 imbalance를 없애기 위해서 2800개씩 sampling한 데이터셋
    - Letter: 대문자, 소문자 알파벳

### TASKS
1. noisy 확률을 찍어가면서 class noise가 classifier의 성능에 얼마나 악영향을 끼치는지 확인.
2. 전체 데이터셋의 일부분(e.g. 10%, 20%, ...)을 여러가지 방식으로 우선순위를 정해 재검토하고 각 방식이 어느정도 성능 향상이 있었는지 확인.
    - 우선순위 방식
        - random하게 일부분을 선택
        - loss가 높은 순으로 일부분을 선택.
        - prediction entropy가 높은 순으로 일부분을 선택.
        - (추후) influence function값이 높은 순으로 일부분을 선택.
    - 성능
        - testing set accuracy
        - $\frac{고쳐진 데이터 수}{검토된 데이터 수}$ 비율
3. (2와 달리) 일절의 추가정보(e.g. 사람에 의한 재검토) 없이 algorithm만으로 Class noise에 robust한 classifier를 구현하고 성능을 찍어봄.
    - Bootstrapping
    - EM algorithm



### TASK 0: Data Analysis

EMNIST balanced dataset을 사용함
총 class 47, 각각 2800장

#### 원본 이미지 예시
![image_samples](https://github.com/sualab/suaWiki/blob/master/noisy-label/images/item_samples.png)

#### 초기 테스트 결과
해당 결과는 외부 noise 없이 원본 데이터를 가지고 학습했을 때 결과값을 나타냄.

- 원본 Paper
    - EMNIST: an extension of MNIST to handwritten letters
    - architecture: OPIUM (extreme learning machine의 변형)
    - accuracy: 78.02%
- Kaggle
    - https://www.kaggle.com/jamilruet13/neural-net-accuracy-85-99
    - architecture: 3-layer MLP with dropout
    - accuracy: 85.99%
- Data Analysis Code
    - suawiki에 기재된 data analysis와 같은 방식으로 진행함.
    - 기존 이미지는 28 x 28인데 이를 224 x 224로 resize한 다음 진행.
    - architecture: Resnet-18
    - accuracy: 89.46%

T-SNE 결과
![t-sne](https://github.com/sualab/suaWiki/blob/master/noisy-label/images/resnet_convnet_embed.jpg)
- 대체적으로 cluster를 잘 형성함.
- 6, b, h, L 등 사람이 보기에도 (글씨체 차이 등을 원인으로) 비슷한 글자들이 서로 비슷한 곳에 몰려 있음.
- 위처럼 사람이 보기에도 구분하기 힘든 글자가 있기 때문에 정확도를 100%에 가깝게 하는 것은 매우 어려움.
- 따라서 중간 중간 noisy label 데이터가 있을 경우 문제가 더 복잡해질 것.

### TASK 1

원본 데이터셋에 1-p확률로 데이터 라벨을 그대로 유지하고, p확률로 기존의 라벨을 제외한 임의의 라벨로 바꿨을 경우 학습된 인공신경망의 error rate가 얼마나 증가했는지 보여주는 실험이다.

다음은 resnet-18에 대해서 진행한 실험 결과이다.
![resnet-18](https://github.com/sualab/suaWiki/blob/master/noisy-label/images/cross_validation_results.png)

30%확률로 라벨이 바꼈을 때, error rate가 약 10%에서 20%로 증가하였다.
하지만 이처럼 무겁고, 224 x 224로 resize해야하는 network를 사용할 경우 학습하는데 지나치게 오랜 시간이 걸리기 때문에 (특정 p에 대해서 한 번 학습하는데 (100epoch) 4시간이 걸림) 이 후 실험은 간소화된 CNN network를 사용하였다.

network architecture는 다음과 같다.
- conv1: 64 kerenel 3x3 stride 1 convolution + relu + 2x2 maxpooling
    - 28x28x1 -> 14x14x64
- conv2: 128 kerenel 3x3 stride 1 convolution + relu + 2x2 maxpooling
    - 14x14x64 -> 7x7x128
- conv3: 256 kerenel 3x3 stride 1 convolution + relu + 2x2 maxpooling
    - 7x7x128 -> 4x4x256
- dense1_dropout: 4096 x 2048 + dropout
    - 4x4x256(4096) -> 2048 -> 2048(dropout)
- logit: 2048 x 47
    - 2048 -> 47
    
그 결과 100 epoch를 돌리는데 약 14분이 걸렸고, accuracy는 약 87.35%가 나왔다. 정확도는 비록 떨어졌지만 다양한 실험을 진행해야만 하기 때문에 좀 더 간소화된 network를 사용하여 뒤의 실험을 진행하고자 한다.

----------
cf)
참고로 인공신경망을 학습하는 과정에서 data에 randomness를 추가하는 것이 noise에 의한 random labeling을 제외하고서 두 가지(__augmentation(random flip)__, __dropout__)가 있다. 

두 방법론은 인공신경망을 학습하는데 __보통__ 도움이 된다고 알려져 있다. 하지만 우리의 경우는 조금 다르다. 몇몇 class에 대해선 EMNIST의 글자에 random flip을 하게 되면 더 이상 맞는 label이 아닐 수도 있기 때문이다. 예를 들어서 4를 horizontal flip하게 되면 그 결과물을 4라고 인식하기에 무리가 있다. 더 나아가서 2를 vertical flip하게 되면 5에 더 가까운 이미지지만 label은 2로 되어있다. 때문에 0이나 8처럼 flip해도 관계없는 class에 대해서는 augmentation이 효과적일 수는 있어도, 몇몇 class에 대해서는 random labeling과 비슷한 효과가 될 수도 있다. 마찬가지로 dropout 때문에 뒷단에 중요한 feature를 날려버리게 되면, (정확한 설명은 아닐 수 있겠지만) b와 6처럼 글자가 서로 비슷한 경우 서로 구분이 안 되어 augmentation과 마찬가지로 labeling 정보가 noisy한 효과를 얻을 수 있다. 결과적으로 이러한 network에 randomness를 추가하는 방법론들을 가지고 학습하게 되면 인공신경망이 robust하게 아니면 오히려 망치는 방향으로 학습이 될 것이다. 이 두 technique이 noisy label에 대해서 정확히 끼치는 영향에 대해서 간단하게 알아보기 위해서 두 실험을 진행했다.
1. dropout을 0으로 고정하고 data augmentation을 진행한 것과 안 한것을 비교
2. data augmenation을 안하고 다양한 dropout rate에 대해서 비교

#### Augmentation
1. dropout: 0 / aug: true
2. dropout: 0 / aug: false

![augmentation](https://github.com/sualab/suaWiki/blob/master/noisy-label/images/augmenation_cross_validation_results.png)

- augmentation의 경우 augmenatation을 한 것과 그렇지 않은 것 사이 특별한 우위를 찾을 수 없었다.
- 때문에 더 단순한 방법론(no augmentation)을 사용하여 뒤 실험을 진행하기로 하였다.

#### Dropout
1. aug: false / dropout: 0.0
2. aug: false / dropout: 0.1
3. aug: false / dropout: 0.5

![dropout](https://github.com/sualab/suaWiki/blob/master/noisy-label/images/dropout_cross_validation_results.png)

- dropout의 경우 dropout rate가 높으면 높을 수록 좋은 결과를 얻을 수 있었다.
- 특히 dropout이 0.5일 때 noise ratio에 좀 더 robust하게 작동하였다.

----------
cf)
Resnet과 Simple CNN 결과 비교.
Resnet과 Simple CNN의 결과를 비교할 때 매우 흥미로운 점이 하나 있다. 바로 간단한 CNN이 noise에 대해서 좀 더 robust하다는 것이다. 아래는 Resnet과 Simple CNN, 그리고 Simple CNN에서 가장 잘 동작하는 세 architecture에 대한 graph이다.
1. Resnet -> aug: true / dropout: x
    - data analysis code과 똑같은 setting이다.
2. Simple CNN -> aug: true / dropout: 0.0
    - 위 resnet과 최대한 비슷한 setting을 하기위해서 data augmentation이 있고 dropout 없는 상황에서 진행하였다.
3. Simple CNN -> aug: false / dropout: 0.5
    - 여태까지 진행했을 때 가장 잘 동작하는 network를 가져와서 performance를 비교하였다.

![architecture](https://github.com/sualab/suaWiki/blob/master/noisy-label/images/architecture_compare.png)

위 그래프를 보면 앞서 설명했듯이 noise가 아예 존재하지 않는 경우에는 Resnet이 가장 잘 동작한다. 이는 Resnet은 간단한 CNN에 대해서 매우 복잡한 구조를 가지고 있어서 상대적으로 많은 weight parameter를 갖고 있고, 덕분에 EMNIST의 세세한 부분까지 관측하여 training set에 대해서 완벽하게 fitting할 수 있기 때문이다. 

하지만 label noise가 들어오게 되면 양상이 달라진다. label noise가 추가될 때 발생하는 최악의 시나리오는 처음 본 데이터셋이 label noise 때문에 다르게 라벨링된 경우이다. 이 경우 해당 이미자와 잘못된 라벨 자체를 network가 통째로 외워버려 틀린 답을 내보내게 될 것이다. Weight 수가 많은 (network capacity가 큰) Resnet은 weight 수가 적어서 전체적으로 뭉뚝그려버리는 CNN과 달리 한 픽셀 한 픽셀을 세밀하게 관측할 수 있기 때문에 조금만 다른 그림이더라도 기존의 것과 구분하여 처음보는 example이라고 생각할 수 있고 이를 라벨과 함께 통째로 외워버리게 되는 경우 error rate가 증가할 가능성이 있다. 

위 그래프를 보면 앞서 언급한 양상을 확인할 수 있다. label noise가 적은 경우 Resnet이 잘 작동하지만, label noise가 커질 경우에는 Resnet이 틀린 example에 대해서 overfitting 되어 error rate가 급상승하게 된다.

우리가 관심갖고 있는 지점은 Noise ratio가 30%인 곳이다. 이 지점에서는 Simple CNN이 복잡한 인공신경망들에 비해서 noise robust하더라도 애초에 network accuracy가 낮기 때문에 Resnet의 error rate가 더 낮다. 하지만 Simple CNN에 dropout을 추가하게 되면 noise ratio가 30%뿐이더라도 Resnet에 비해서 더 잘 동작하는 classifier를 얻을 수 있다. 이는 architecture를 바꾸는 것만으로 성능을 향상시킬 수 있다는 것이다.

------------

이후 시뮬레이션에서는 augmentation 안하고, dropout은 추가한 network를 사용하여 진행할 것이다. 또 label의 noise가 30% 정도 추가되었을 때 괄목할만한 변화(resnet과 dropout을 하지 않은 network가 약 2배정도 error rate이 증가했음)가 관측되었기 때문에 30%를 고정으로 이후 task(noise가 정해졌을 때 성능을 향상시키는 방법론 고찰)를 진행하고자 한다.

### TASK 2

labeling이 정확하지 않다고 생각했을 때 우리가 취할 수 있는 가장 간단한 방법은 데이터셋을 재검토하는 것이다. 우리에게 친숙한 숫자와 알파벳으로 구성되어 있는 EMNIST dataset의 특성상 우리는 image를 눈으로 확인하는 걸로 정확한 label을 찾을 수 있다. 때문에 재검토된 대상은 앞선 noise 때문에 label이 바뀌었더라도 재검토하는 것으로 정확한 label을 복원할 수 있다. 하지만 데이터셋이 지나치게 크기 때문에 모든 데이터셋을 재검토하는 것은 사실상 불가능하다. 때문에 데이터의 일부분을 검토했을 때 얼마만큼 성능향상이 있었는지 확인하고자 한다. 좀 더 구체적으로 말하자면 이번 실험에서는 특정 measure(random, loss, entropy)에 따라서 training set은 정렬하고, 그 기준으로 0% 5% 10% 15% 20% 25% 30% 재검토 했을 때 변화한 performance를 확인할 것이다.

재검토 후 classifier의 performance 향상을 확인해보기 앞서 특정 measure를 사용하여 재검토했을 때 얼마만큼 라벨이 고쳐졌는지, 비율을 확인하는 것이다. 다음은 전체 데이터에서 10%, 20%, 30%, ..., 90%, 100%를 각각 measure에 따라서 우선순위로 재검토했을 때 전체 잘못 라벨링된 데이터 중에서 몇 퍼센트를 찾아냈는지 보여주는 그래프이다.

![recovery](https://github.com/sualab/suaWiki/blob/master/noisy-label/images/recovery%20results.png)

당연히 0%를 재검토했을 때와 100%를 재검토했을 때에는 우선순위가 상관없기 때문에 서로 같은 값을 가진다. 중요하게 봐야할 지점은 그 사이인데, random하게 재검토를 하는 것보다 loss나 entropy가 높은 순으로 재검토를 하게 될 경우 mislabel된 example을 찾을 확률이 높을 걸 확인할 수 있다. 수치적으로 이야기하자면 전체 mislabel된 데이터중에서 50%를 고치기 위해서는 random하게 재검토했을 경우는 전체 데이터중에서 50%를 봐야하지만, entropy가 높은 순으로 재검토했을 경우에는 46.5%를, loss가 높은 순으로 재검토했을 경우에는 46.9%만 보고서도 할 수 있었다. (하지만 이는 기대한 것에 비해서 향상된 정도가 낮다.) 그 외에 전체적으로 __고쳐진 데이터 수로만 비교했을 때에는__ entropy가 loss보다 아주 조금 우세하였다.

하지만 이 비율이 classifier의 performance에 정확히 비례하는 것은 아니다. 그 이유는 하나의 training example이 neural network에 끼치는 영향이 다를 수 있기 때문이다. 따라서 제대로 된 성능을 측정하기 위해선 위와 같이 재검토된 데이터를 가지고 다시 학습시켰을 때 얻을 수 있는 error rate를 확인해야 한다.

먼저 0% 5% 10% 15% 20% 25% 30%를 재검토한 뒤, 새로운 라벨을 가지고 새롭게 네트워크를 학습했을 때 error rate을 찍었을 경우 다음과 같은 결과를 얻을 수 있었다.

![review and retrain 0.3](https://github.com/sualab/suaWiki/blob/master/noisy-label/images/review_retrain_0.0_0.3.png)
__아직 entropy에 대한 값을 뽑지 않았음. 초록선이 random을 의미함__
Random하게 재검토한 것에 비해서 loss가 높은 순으로 제검토하게 되면, 틀린 data의 loss가 낮아져 더 안정적으로 그리고 높은 정확도로 학습이 될 것이라고 예측하였으나, 결과값을 보면 서로 엎치락 뒤치락만 할 뿐 우열을 가리기 힘들었다. 오히려 loss를 우선순위로 재검토했을 경우 더 불안정한 결과값을 나타내었다. 5%를 단위로, 0%~30%의 범위에선 심지어 error rate가 감소하는 양상도 결과값의 불안정성 때문에 믿기 힘든 상황이었기에 10%를 단위로, 0%~100%의 범위를 재검토하며 다시 재학습을 시켜보았다.

![review and retrain 1.0](https://github.com/sualab/suaWiki/blob/master/noisy-label/images/review_retrain_0.0_0.3.png)
재검토된 비율이 높을 수록 당연히 error rate가 감소하는 것을 관측할 수 있었다. 하지만 여전히 random policy와 loss policy사이 __눈에 띄는 우열을 찾기는 힘들었다__.

#### Conclusion

- 좀 더 다양한 measure에 대해서 해볼 것.
- 하지만 influence function에서 나온 논문만큼 dramatic한 향상은 보기 힘들었음.
- 큰 향상이 없었다는 것은 그만큼 악의적인 label distortion이 없었다는 것.
- 마찬가지로 정도가 data specific할 것.
    - 특히 데이터셋의 크기에도 큰 영향이 있을 것.
    - 데이터셋 크기가 커서 노이즈가 있어도 주변 비슷한 그림들을 모아서 보면 noise가 상쇄될 수 있음.
    - 데이터셋의 크기가 작아서 overfitting문제가 심각해질 경우 효과는 dramatic해질 수도 있음.
- 데이터셋 크기가 클 경우, error rate을 약 1%감소시키기 위해서 30%의 데이터를 재검토해야하는데 이는 몇 만장 단위로 쉽지 않음. (그렇게 효율적인 방법론은 아닐 수 있음.)


### TASK 3

Task 2에서는 우선순위를 정한 뒤 데이터의 일부분을 재검토하는 것이 label noise에 의한 성능 하락을 얼마만큼 완화시켜줄 수 있을지 확인하였다. 하지만 이 경우, 라벨이 noisy하여 현 라벨을 믿을 수 없더라도 oracle이 존재해서 해당 이미지를 보면 정확한 라벨을 복원할 수 있다는 가정이 있었다. 데이터를 일일이 재검토하는 것은 추가적인 resource가 드는 것이기 때문에 적은 데이터를 재검토하는 것으로도 큰 gain이 있어야만 한다.

하지만 위에서 확인해보았듯이 눈에 띄는 향상을 얻기 위해서는 대부분의 데이터를 재검토해야하고 (적은 단위, 10%, 20% 등에서는 큰 향상이 없었음), 이는 EMNIST 등 딥러닝에 쓰일만큼 큰 데이터셋에서는 수만장 단위이다. 사람이 수 만장을 일일이 재검토하는 것은 수 만장이 전체 데이터셋의 일부분이더라도 여전히 힘든 일이다. 때문에 이 섹션에서는 별도의 human resource없이 label noise문제를 해결할 수 있는 방법을 알아보고 벤치마크를 확인해볼 것이다.



#### Boot-strapping
refer: Training deep neural networks on noisy labels with boot-strapping (ICLR 2015)

이 논문은 boot-strapping을 통해서 label noise를 줄이고자 함. 간단하게 설명하자면 기존의 noisy한 라벨과 현재 network의 예측 값을 사용해서 다시 라벨링하고, 이 새로운 라벨을 학습에 사용함.

좀 더 자세히 설명하면 다음과 같다.

- x: input feature
- y: ground truth label
- z: observed noisy label
    - 라벨 z는 onehot vector이다. i.e., $\mathbf{z} = (z_1, z_2, ..., z_L)$
    
우리가 가지고 있고 학습에 쓸 데이터셋은 $\{(\mathbf{x}^{(i)},\mathbf{z}^{(i)})\}_{i=1}^{N}$ 

기존의 cross entropy loss는 다음과 같이 쓸 수 있다.
\begin{equation*}
L_{original}(\mathbf{x}^{(i)},\mathbf{z}^{(i)}) = \sum_{k=1}^{L}{z_k^{(i)} \log{P_{\theta}(z = z_k^{(i)} \vert \mathbf{x}^{(i)})}}
\end{equation*}

기존의 noisy label (z) 대신 새 label ($q(\mathbf{x})$)을 넣으면, i.e. $\mathbf{z} \leftarrow \beta \mathbf{z} + (1-\beta) \mathbf{q}(\mathbf{x})$ 다음과 같은 loss를 얻을 수 있다.

\begin{equation*}
L(\mathbf{x}^{(i)},\mathbf{z}^{(i)}) = \sum_{k=1}^{L}{(\beta z_k^{(i)} + (1-\beta) q_k(\mathbf{x}^{(i)})) \log{P_{\theta}(z = z_k^{(i)} \vert \mathbf{x}^{(i)})}}
\end{equation*}

이때 함수 q를 다양한 방법으로 설정할 수 있는데, 이 논문에서 제시한 방법은 2개이다.
1. $\mathbf{q}_{soft}(\mathbf{x}) = P_{\theta}(\mathbf{z}\vert \mathbf{x})$
2. $\mathbf{q}_{hard}(\mathbf{x}) = \mathbb{1}[k = \arg{\max{P_{\theta}(z = z_k \vert \mathbf{x})}}]$

(1)은 network prediction의 soft label, (2)는 network prediction의 hard label이라고 생각할 수 있다.
새롭게 제시한 두 라벨을 가지고 loss function을 정의하면 다음과 같다.

1. $L_{soft}(\mathbf{x}^{(i)},\mathbf{z}^{(i)}) = \sum_{k=1}^{L}{(\beta z_k^{(i)} + (1-\beta) P_{\theta}(z = z_k^{(i)} \vert \mathbf{x}^{(i)})) \log{P_{\theta}(z = z_k^{(i)} \vert \mathbf{x}^{(i)})}}$
2. $L_{hard}(\mathbf{x}^{(i)},\mathbf{z}^{(i)}) = \sum_{k=1}^{L}{(\beta z_k^{(i)} + (1-\beta) q_{hard,k}(\mathbf{x}^{(i)})) \log{P_{\theta}(z = z_k^{(i)} \vert \mathbf{x}^{(i)})}}$

cf) $L_{soft}$의 경우 entropy regularized cross entropy term과 같음을 확인할 수 있다. prediction entropy가 minimize 되는 방향으로 학습이 될 텐데 그 결과 network prediction이 high confidence가 되게 될 것이다. i.e. 하나의 라벨에 대해서 sharp하게 될 것. 

추가 견해를 붙이자면 label이 noisy할 경우, 데이터가 충분히 많다면 같은 이미지더라도 여러번 관측될 수 있다. 때문에 같은 이미지더라도 서로다른 라벨로 관측될 것이다. (deep-learning-is-robust-to-massive-label-noise 문제상황과 비슷해짐) 그 결과 각 image가 smoothed된 label을 갖는 효과가 될 것이다. entropy regularizer term을 추가한다면 이를 다시 sharp하게 만드는 효과를 줄 수 있을 것.

cf) $L_{hard}$의 경우는 prediction 후 라벨을 linear combination 한 것. hard prediction 값을 gradient에 넣을 지 뺄 지 값을 비교해 볼 것. (soft 때는 넣어서 entropy minimization효과를 얻었음)

cf) 여기서는 network prediction을 training step 단위로 (freeze하지 않고) 진행했음. 보통의 boot strapping은 안정성 등의 문제로 freezed network output을 사용함. 또 초창기 network output은 data와 관계없이 uniform할 것이기 때문에 오히려 label에 uniform noise를 추가한 효과만 줄 것이기 때문에 오히려 방해가 될 수 있음. 따라서 더 해볼 수 있는 것으로는 
1. inital point를 noisy label로 학습한 network output을 사용
2. freeze된 network를 만들어서 해당 prediction output을 사용
등이 있을 것.

cf) MNIST dataset에 대해서 실험을 진행할 때 hard version은 $\beta = 0.8$로, soft version은 $\beta = 0.95$로 진행했을 때 가장 잘 작동한다고 했음. 때문에 EMNIST에서 진행할 때에는 이와 비슷한 값인 0.9로 고정하여 진행하였음.

# 실험 결과
- 진행중..


#### EM algorithm
refer: 
- Training deep neural-networks based on unreliable labels (ICASSP 2016)
- Training deep neural-networks using a noise adaptation layer (ICLR 2017)

noisy channel model
hidden variable can be found by EM algorithm

관측된 label이 y로부터 생성되었다고 가정한다면, noisy channel로 modeling할 수 있다. (그리고 위 상황처럼 y에서 임의 확률로 flip하는 경우라면 이 가정을 만족한다.) 그 결과 우리는 feature vector x, true label y, observed label z를 markov chain X - Y - Z으로 생각할 수 있다. 우리의 목적은 X와 Z를 관측하여 latent variable Y를 찾는 것이다. 이러한 문제는 EM algorithm을 통해서 해결할 수 있다.

Network modeling을 다음과 같이 하였다. 기존의 classification 문제처럼 input feature X를 neural network를 거쳐서 y를 얻고, y에서 z로 넘어가는 noisy channel의 확률값은 y,z 모두 categorical이기 때문에 matrix로 mapping하였다.

![network](https://github.com/sualab/suaWiki/blob/master/noisy-label/images/noisy%20channel.png)

우리가 하고자 하는 것은 관측된 데이터셋, $\{(x^{(i)},z^{(i)})\}_{i=1}^{N}$를 model에 맞추고, 이 때 얻은 parameter를 이용해서 latent variable y를 찾는 것이다. 이를 기반으로 loss function을 만들면 다음과 같다.

\begin{eqnarray*}
\min_{w}{\min_{\theta}{L(w,\theta)}} 
&=& -\sum_{t=1}^{N}{\log{P(z^{(t)}\vert x^{(t)} ; w, \theta)}} \\
&=& -\sum_{t=1}^{N}{\log{(\sum_{i=1}^{L}{P(z^{(t)}\vert y^{(t)}=i ; \theta)P(y^{(t)}\vert x^{(t)} ; w)})}}
\end{eqnarray*}

- $\theta \rightarrow$ noise distribution을 학습
- $w \rightarrow$ likelihood function을 최대화

위 loss로부터 variational inference를 사용하면 EM algorithm 형태를 얻을 수 있음.

refer: Bishop, Christopher M. Pattern recognition and machine learning. Vol.4. No.4

cf) Derivation of EM algorithm

\begin{eqnarray*}
\log{p(z_t \vert x_t ; w, \theta)}
&=& \sum_{i=1}^{k}{p(y_t=i \vert z_t,x_t ; w_0,\theta_0) \log{p(z_t \vert x_t ; w,\theta)}}\\
&=& \sum_{i=1}^{k}{p(y_t=i \vert z_t,x_t ; w_0,\theta_0) \log{\frac{p(y_t=i, z_t \vert x_t ; w,\theta)}{p(y_t=i \vert z_t , x_t ; w,\theta)}}}\\
&=& \sum_{i=1}^{k}{p(y_t=i \vert z_t,x_t ; w_0,\theta_0) \left( \log{\frac{p(y_t=i, z_t \vert x_t ; w,\theta)}{p(y_t=i \vert z_t , x_t ; w_0,\theta_0)}} + \log{\frac{p(y_t=i \vert z_t , x_t ; w_0,\theta_0)}{p(y_t=i \vert z_t , x_t ; w,\theta)}} \right)}\\
&=& \sum_{i=1}^{k}{p(y_t=i \vert z_t,x_t ; w_0,\theta_0) \log{\frac{p(y_t=i, z_t \vert x_t ; w,\theta)}{p(y_t=i \vert z_t , x_t ; w_0,\theta_0)}}} + D_{KL}(p_{w_0,\theta_0} \vert\vert p_{w,\theta})\\
&\ge& \sum_{i=1}^{k}{p(y_t=i \vert z_t,x_t ; w_0,\theta_0) \log{\frac{p(y_t=i, z_t \vert x_t ; w,\theta)}{p(y_t=i \vert z_t , x_t ; w_0,\theta_0)}}}\\
&=& \sum_{i=1}^{k}{p(y_t=i \vert z_t,x_t ; w_0,\theta_0) \log{p(y_t=i, z_t \vert x_t ; w,\theta)}} + \text{const}\\
&=& \sum_{i=1}^{k}{p(y_t=i \vert z_t,x_t ; w_0,\theta_0) \left( \log{p(y_t=i \vert x_t ; w)} + \log{p(z_t \vert y_t=i ; \theta)}\right)} + \text{const}\\
&\because& \text{X-Y-Z Markov chain}
\end{eqnarray*}

위 loss function은 w와 $\theta$에 대해서 각각 나눠지기 때문에 w에 대해서, $\theta$에 대해서 update를 따로 하면 된다.

전체적인 algorithm은 다음과 같다.

![overall algorithm](https://github.com/sualab/suaWiki/blob/master/noisy-label/images/overall%20algorithm.png)

cf)
- E-step (Estimation)
    - \begin{eqnarray*}
    c_{t,i} 
    &=& p(y_t = i \vert x_t, z_t ; w_0, \theta_0) \\
    &=& \frac{p(z_t\vert y_t=i; \theta_0) p(y_t = i \vert x_t; w_0)}{\sum_{j}{p(z_t\vert y_t=j; \theta_0) p(y_t = j \vert x_t; w_0)}}
    \end{eqnarray*}
    - $w_0, \theta_0$가 주어졌을 경우 위의 방법을 이용하면 latent variable의 posterior probability를 구할 수 있다.
- M-step (Maximization)
    - $\theta$
        - y, z 모두 categorical이기 때문에 confusion matrix로 noisy channel을 만들 수 있다.
        - loss function을 풀면 closed form solution을 얻을 수 있다.
    - $w$
        - w의 경우 neural network이기 때문에 고정된 값 $c_{t,i}$를 이용해서 loss를 얻고 이를 SGD로 optimize하면 된다.
        
        
# 실험결과
- 구현중

# 면접용 modification

- noise 수가 많으니까 재검토 방법이 너무 비효율적임. (몇 만장을 고쳐야 정확도가 1,2% 오름)
- influence function 사용해서 adversarial attack함?



추가 논문 참고
- Learning to label aerial images from noisy data
    - https://www.cs.toronto.edu/~vmnih/docs/noisy_maps.pdf (ICML2012)
- Training deep neural networks on noisy labels with boot-strapping (ICLR 2015 workshop)
- ... unreliable data
- ... noisy data (ICLR 2017)
- How do neural network overcome label noise?
    - https://openreview.net/pdf?id=ryu4RYJPM (ICLR 2018 workshop)
- ChoiceNet: robust learning by revealing output correlations
    - mixture of target distribution & out-of-distribution 
    - kakao brain 